In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
pd.set_option('display.max_colwidth', None)

## Explore Dataset


In [ ]:
train_df.head()

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df[train_df['target'] > 0].head()

In [ ]:
print(train_df['question_text'].isnull().sum())

[](http://)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [ ]:
import keras

In [ ]:
train_df.head()

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(train_df['question_text'],
                                                      train_df['target'],
                                                      test_size=0.25,
                                                      random_state=42)


test_x = test_df['question_text']

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000)
tokenizer.fit_on_texts(train_x)
train_x = tokenizer.texts_to_sequences(train_x)

valid_x = tokenizer.texts_to_sequences(valid_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [ ]:
print(train_x[0:10])

In [ ]:
max_length = 50

In [ ]:
valid_x = tf.keras.preprocessing.sequence.pad_sequences(valid_x, maxlen=max_length)
train_x = tf.keras.preprocessing.sequence.pad_sequences(train_x, maxlen=max_length)
test_x = tf.keras.preprocessing.sequence.pad_sequences(test_x, maxlen=max_length)

In [ ]:
inputs = keras.Input(shape=(max_length,))
x = keras.layers.Embedding(50000, 256)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(64))(x)
x = keras.layers.Dense(32, activation="sigmoid")(x)
x = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.models.Model(inputs = inputs, outputs = x)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(train_x, train_y, batch_size=512, epochs=2, validation_data=(valid_x, valid_y))

In [ ]:
result = model.predict([test_x], batch_size=1024, verbose=1)
print(result)

In [ ]:
result[result >= 0.5] = True
result[result < 0.5] = False
result = result.astype(np.int64)

In [ ]:
submit_df = pd.DataFrame()
submit_df['qid'] = test_df['qid']
submit_df['prediction'] = result

In [ ]:
submit_df.to_csv('submission.csv', index=False)